In [8]:
import numpy as np
import pathlib
from PIL import Image, Image
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tqdm import tqdm
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
SEED = 78
img_size = 100
data_dir = pathlib.Path("DATA")

In [14]:
model = Sequential([
  # Normalize
  layers.Rescaling(
    1./255, 
    input_shape=(img_size, img_size, 3)
  ),
  # Layers
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  # 3x3 max pooling instead of 2x2 to cut down on parameters
  layers.MaxPooling2D(3),
  layers.Conv2D(32, 3, padding='same', activation='tanh'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, padding='same', activation='tanh'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(256, activation='relu'),
  layers.Dense(33)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Faster to load weights instead of the entire model
model.load_weights(r"aksov19_lmach19_loss-0.2077_accuracy-0.9624.h5")

In [15]:
predictions = pd.DataFrame(columns=["path", "prediction"])

for path in list(data_dir.glob("*")):
    img = tf.keras.utils.load_img(
        path, target_size=(img_size, img_size)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    prediction = model.predict(img_array)
    score = tf.nn.softmax(prediction[0])

    predictions = predictions.append({"path" : path, "prediction" : np.argmax(score)+1}, ignore_index=True)

predictions

,path,prediction
0,test_DATA\f1.jpg,21
1,test_DATA\f2.jpg,21
2,test_DATA\f3.png,21
3,test_DATA\f4.jpg,21


In [16]:
predictions.to_csv("aksov19_lmach19.csv")